# EMISSOR chat bot with audio backend

### UNDER CONSTRUCTION 

## Running with ASR

In [ ]:
import numpy as np
from cltl.asr.speechbrain_asr import SpeechbrainASR
from cltl.backend.source.pyaudio_source import PyAudioSource
from cltl.vad.webrtc_vad import WebRtcVAD

from cltl.backend.api.util import raw_frames_to_np

In [ ]:
source = PyAudioSource(16000, 1, 480)
vad = WebRtcVAD(allow_gap=250, padding=5)

# Store detected audio for debugging
# asr = SpeechbrainASR(model_id="speechbrain/asr-transformer-transformerlm-librispeech", sampling_rate=16000, storage="wav")
asr = SpeechbrainASR(model_id="speechbrain/asr-transformer-transformerlm-librispeech")

    
def detect_and_transcribe():
    with source as audio:
            print("---------   Listen   ---------")
            frames = raw_frames_to_np(audio, source.frame_size, source.channels, source.depth)
            speech, offset, consumed = tuple(vad.detect_vad(frames, source.rate))
            print("--------- Transcribe --------")
            text = asr.speech_to_text(np.concatenate(tuple(speech)), source.rate)
            print("--------- Detected:", text)

            return text


In [ ]:
for _ in range(5):
        try:
        detect_and_transcribe()
    except Exception as e:
        print("Failed", e)

print("Calibrated")

In [ ]:
while True:
    try:
        detect_and_transcribe()
    except Exception as e:
        print("Failed", e)